In [3]:
import findspark

import pandas
import plotly as p
import plotly.plotly as py
from plotly.graph_objs import *
import plotly.figure_factory as ff
import plotly.plotly as py
import plotly.graph_objs as go
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from plotly.widgets import GraphWidget

import matplotlib.pyplot as plt  # mathplotlib

findspark.init("C:\spark-2.2.1-bin-hadoop2.7")
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.types import *
sparkSession = SparkSession.builder.appName(
    "option-pricer-write-to-hadoop").getOrCreate()

resultsHiveDF = sparkSession.read.format('json').load('E:\ProjectDB')

resultsHiveDF.createOrReplaceTempView("optionData")

<IPython.core.display.Javascript object>

In [144]:
tickers = sparkSession.sql("SELECT Ticker FROM optionData").collect()
tickerSymbols = []
for i in tickers: # remove Row(Ticker=) from ticker
   tickerSymbols.append(i['Ticker'])

expDate = sparkSession.sql("SELECT ExpirationDate FROM optionData WHERE Ticker Like 'MMM'").collect()

riskFreeRates = sparkSession.sql("SELECT riskFreeRates FROM optionData WHERE Ticker Like 'MMM'").collect()

strikePrices = sparkSession.sql("SELECT Prices['StrikePrice'] FROM optionData WHERE Ticker Like 'ZTS'").collect()

numOfDays = sparkSession.sql("SELECT NumberOfDays FROM optionData WHERE Ticker Like 'ABBV'").collect()



In [118]:
# Cell to declare all funcions

# Function to get prices based on a specific ticker symbol
def getOptionPricesByTicker(symbol):
    return sparkSession.sql("SELECT Prices FROM optionData WHERE Ticker Like '" + symbol + "'").collect()

# Function to turn int to array of ints for number of days before expDate
# eg. number of says is 5 returns [1,2,3,4,5]
def getNumberDaysArray(x):
    numDaysArr = []
    for i in range(1, x[0]['NumberOfDays'] + 2):
        numDaysArr.append(i)
    return numDaysArr

def getPricesByStrike(optionPrices, strike):
    allPrices = []
    for i in optionPrices:
        if i!= None and i['StrikePrice'] == strike:
            allPrices.append(i['Put'])
    for i in allPrices:
        if i is None:
            allPrices.remove(i)
    for i in allPrices: # for some reason have to run twice as it skips last None in above 
        if i is None:
            allPrices.remove(i)
    return allPrices



def getPricesByStrikeAndRate(optionPrices, strike, RiskFreeRate):
    allPrices = []
    for i in optionPrices[0][0]:
        if i != None and i['StrikePrice'] == strike and i['RiskFreeRate'] == RiskFreeRate:
            allPrices.append(i[0])
    return allPrices

def getCalls(optionPrices):
    callPrices = []
    for i in optionPrices[0][0]:
        if i['Call'] != None:
            callPrices.append(i['Call'])
    return callPrices

def getPuts(optionPrices):
    putPrices = []
    for i in optionPrices[0][0]:
        if i['Put'] != None:
            putPrices.append(i['Put'])
    return putPrices

def getCalls2(optionPrices):
    callPrices = []
    for i in optionPrices[0][0]:
        if i['Call'] != None:
            callPrices.append(i)
    return callPrices

def getPuts2(optionPrices):
    putPrices = []
    for i in optionPrices[0][0]:
        if i['Put'] != None:
            putPrices.append(i)
    return putPrices

def priceToArray(opPrices):
    prices = {}
    count = 1
    for i in opPrices:
        priceArr = []
        for x in i:
            if x is not None:
                priceArr.append(x)
        prices['opt'+str(count)] = (priceArr.copy())
        count = count + 1
    return prices


def riskRateToArray(rates):
    riskRatesArray = []
    for i in rates[0]['riskFreeRates']:
        riskRatesArray.append(i)
    return riskRatesArray

def getSinglePut(optionPrices):
    putPrices = []
    for i in optionPrices[0]:
        if i != None:
            putPrices.append(i)
    return putPrices

def getSingleCall(optionPrices):
    putPrices = []
    for i in optionPrices[0]:
        if i != None:
            putPrices.append(i)
    return putPrices

def getStrikePriceUnique(prices):
    strikePricesArray = []
    for i in prices[0]['Prices.StrikePrice']:
        if i != None and i not in strikePricesArray:
            strikePricesArray.append(i)
    return strikePricesArray 

def getRiskRateForGraph(ticker): #Probably have to change if more than 2 lines as graph code might crash TEST THIS
    ratesArr = []
    count = 1
    riskFreeRates = sparkSession.sql("SELECT riskFreeRates FROM optionData WHERE Ticker Like '" +
                                     ticker + "'").collect()
    for rate in riskFreeRates[0]['riskFreeRates']:
        ratesArr.append(rate)
    return ratesArr


In [119]:
# def getPricesByRiskFreeRate(optionPrices, RiskFreeRate):
#     allPrices = []
#     for i in optionPrices[0][0]:
#         if i != None and i['RiskFreeRate'] == RiskFreeRate:
#             allPrices.append(i[0])
#     return allPrices

# def priceToDic(opPrices):
#     prices = {}
#     count = 1
#     for i in opPrices:
#         priceArr = []
#         for x in i:
#             if x != "None":
#                 priceArr.append(x)
#         prices['opt'+str(count)] = (priceArr.copy())
#         count = count + 1
#     return prices


In [22]:
prices = getOptionPricesByTicker('ABBV')

callPrices = []
putPrices = []

#callPricesStrike = getPricesByStrike(prices, 65.0)



#callPricesSR = getPricesByStrikeAndRate(prices, 67.5, 1.67)

callPrices = getCalls(prices)
callPricesRate = priceToDic(getPricesByRiskFreeRate(prices, 1.67))
#print(callPricesRate )

putPrices = getPuts(prices)
        
numberOfDays = getNumberDaysArray(numOfDays)
    

ValueError: StrikePrice

In [218]:
def getOptionPricesByTicker(symbol):
    return sparkSession.sql("SELECT Prices FROM optionData WHERE Ticker Like '" + symbol + "'").collect()

print(getOptionPricesByTicker('AMZN'))

[Row(Prices=[Row(Call=Row(2018-03-14=0.0, 2018-03-15=104.8838889696778, 2018-03-16=2187018.9338098704, 2018-03-17=None, 2018-03-18=None, 2018-03-19=None, 2018-03-20=None, 2018-03-21=None, 2018-03-22=None, 2018-03-23=None, 2018-03-24=None, 2018-03-25=None, 2018-03-26=None, 2018-03-27=None, 2018-03-28=None, 2018-03-29=None, 2018-03-30=None, 2018-03-31=None, 2018-04-01=None, 2018-04-02=None, 2018-04-03=None, 2018-04-04=None, 2018-04-05=None, 2018-04-06=None, 2018-04-07=None, 2018-04-08=None, 2018-04-09=None, 2018-04-10=None, 2018-04-11=None, 2018-04-12=None, 2018-04-13=None, 2018-04-14=None, 2018-04-15=None, 2018-04-16=None, 2018-04-17=None, 2018-04-18=None, 2018-04-19=None, 2018-04-20=None), Put=None, RiskFreeRate=1.67, StrikePrice=640.0, Volatility=1.0000000000000003e-05), Row(Call=Row(2018-03-14=0.0, 2018-03-15=887.9328105849337, 2018-03-16=13583162.624596022, 2018-03-17=None, 2018-03-18=None, 2018-03-19=None, 2018-03-20=None, 2018-03-21=None, 2018-03-22=None, 2018-03-23=None, 2018-03-

In [120]:
# Function to get prices based on a specific ticker symbol
def getOptionPricesByTicker(symbol):
    return sparkSession.sql("SELECT Prices FROM optionData WHERE Ticker Like '" + symbol + "'").collect()

def getCalls(optionPrices):
    callPrices = []
    for i in optionPrices[0][0]:
        if i['Call'] != None:
            callPrices.append(i['Call'])
    return callPrices

def getPuts(optionPrices):
    putPrices = []
    for i in optionPrices[0][0]:
        if i['Put'] != None:
            putPrices.append(i['Put'])
    return putPrices

def getCalls2(optionPrices):
    callPrices = []
    for i in optionPrices[0][0]:
        if i['Call'] != None:
            callPrices.append(i)
    return callPrices

def getPuts2(optionPrices):
    putPrices = []
    for i in optionPrices[0][0]:
        if i['Put'] != None:
            putPrices.append(i)
    return putPrices

def priceToArray(opPrices):
    prices = {}
    count = 1
    for i in opPrices:
        priceArr = []
        for x in i:
            if x is not None:
                priceArr.append(x)
        prices['opt'+str(count)] = (priceArr.copy())
        count = count + 1
    return prices

In [216]:
def graphCallsByTicker(tickerSymbol):
    opPriceDic = getOptionPricesByTicker(tickerSymbol)
    pp = getCalls(opPriceDic)
    ppDic = priceToArray(pp)
    numOfDays = sparkSession.sql("SELECT NumberOfDays FROM optionData WHERE Ticker Like '" + tickerSymbol + "'").collect()
    numberOfDays = getNumberDaysArray(numOfDays)
    for key, values in ppDic.items():
        plt.plot(numberOfDays, values)
    plt.xlabel('Days to Expiration')
    plt.ylabel('Option Prices')
    plt.title('Calls Options for '+ tickerSymbol)
    plt.ticklabel_format(style='plain')
    plt.show()

In [217]:
interact(graphCallsByTicker, tickerSymbol=tickerSymbols);

interactive(children=(Dropdown(description='tickerSymbol', options=('AMZN', 'GOOGL', 'ROK', 'GOOG', 'PCLN', 'MCHP', 'SPG', 'FDX', 'NTRS', 'AZO', 'NFLX', 'AAPL', 'CMG', 'CB', 'BA', 'AMAT', 'MU', 'VMC', 'NVDA', 'STX', 'CRM', 'WMT', 'CELG', 'AGN', 'GILD', 'BIIB', 'ADSK', 'CHTR', 'GPN', 'TGT', 'MSFT', 'MHK', 'NTAP', 'C', 'LOW', 'ABC', 'WDC', 'ISRG', 'DAL', 'JPM', 'LRCX', 'LMT', 'REGN', 'INTC', 'HCN', 'PYPL', 'BBY', 'FB', 'VLO', 'CAT', 'UAL', 'ULTA', 'GM', 'QCOM', 'AVGO', 'XOM', 'GS', 'ILMN', 'WYNN', 'MO', 'V', 'BLK', 'UNP', 'ORCL', 'ATVI', 'JNJ', 'SCHW', 'CMCSA', 'KSS', 'ABBV', 'RHT', 'SLB', 'ALGN', 'CVX', 'CLX', 'ADI', 'MGM', 'DE', 'COF', 'NKE', 'DLTR', 'PG', 'DG', 'NEM', 'AXP', 'CVS', 'DVN', 'WFC', 'CBOE', 'MCD', 'HD', 'EA', 'ESRX', 'TXN', 'ADBE', 'MNST', 'UPS', 'NOC', 'SHW', 'GE', 'URI', 'VZ', 'RCL', 'HCA', 'COST', 'CSCO', 'MMM', 'UNH', 'XLNX', 'DWDP', 'UTX', 'F', 'NSC', 'DIS', 'EXPE', 'TRIP', 'DISH', 'IBM', 'GPS', 'LUV', 'HAL', 'APC', 'CBS', 'AIG', 'BMY', 'AAL', 'NWL', 'PXD', 'CAH', 'APA', 'SWKS', 'MDLZ', 'KO', 'AMGN', 'HON', 'SYF', 'BAC', 'MOS', 'HAS', 'SBUX', 'ANTM', 'GWW', 'M', 'NUE', 'EFX', 'T', 'PEP', 'AKAM', 'PM', 'ANDV', 'EBAY', 'HUM', 'MAR', 'WBA', 'MRK', 'GRMN', 'KSU', 'LYB', 'TSCO', 'EOG', 'AMT', 'HES', 'ABT', 'AMD', 'TIF', 'CMI', 'INCY', 'KR', 'IP', 'MS', 'MDT', 'CME', 'OXY', 'PFE', 'MPC', 'KORS', 'BDX', 'PSX', 'FCX', 'ALXN', 'EQIX', 'CSX', 'GD', 'PCG', 'DFS', 'CCL', 'ETFC', 'COP', 'WHR', 'MET', 'DOV', 'CTL', 'ADM', 'HOG', 'GLW', 'EMR', 'ITW', 'NRG', 'DHI', 'MA', 'TROW', 'VFC', 'TWX', 'LLY', 'FFIV', 'CI', 'ETN', 'TAP', 'USB', 'KMI', 'RRC', 'HSY', 'CF', 'RL', 'MYL', 'VIAB', 'CTSH', 'YUM', 'AFL', 'SPGI', 'MCK', 'VRTX', 'FL', 'KMX', 'NOV', 'STZ', 'MON', 'TXT', 'TMO', 'CL', 'NFX', 'TSN', 'BK', 'ZTS', 'GPC', 'EW', 'CNC', 'HOLX', 'CXO', 'KMB', 'OKE', 'DHR', 'ADP', 'PPG', 'HP', 'SWK', 'ED', 'PSA', 'AAP', 'PH', 'BAX', 'XL', 'PDCO', 'HIG', 'MRO', 'BBT', 'ALB', 'FLR', 'CBG', 'GT', 'SYK', 'HRL', 'KLAC', 'PX', 'PRGO', 'PRU', 'WMB', 'MAT', 'PHM', 'ADS', 'HPQ', 'BHGE', 'ECL', 'AMG', 'BSX', 'PVH', 'IR', 'UAA', 'TPR', 'AMP', 'ACN', 'BRK.B', 'SNA', 'APD', 'QRVO', 'NCLH', 'SBAC', 'CTXS', 'EMN', 'ORLY', 'KEY', 'NDAQ', 'CAG', 'ARNC', 'AET', 'JNPR', 'RTN', 'SO', 'ICE', 'CERN', 'HPE', 'EQT', 'PAYX', 'FISV', 'VRSN', 'DLR', 'JCI', 'XEC', 'SYMC', 'COTY', 'K', 'COG', 'CPB', 'MTD', 'INTU', 'AYI', 'CDNS', 'AIZ', 'MLM', 'KHC', 'EVHC', 'TDG', 'NLSN', 'WY', 'UA', 'ZBH', 'NEE', 'ETR', 'CINF', 'FOXA', 'ALK', 'SEE', 'SNI', 'BXP', 'TJX', 'PWR', 'PNC', 'IDXX', 'HLT', 'JWN', 'FMC', 'O', 'CHRW', 'SYY', 'HRB', 'FE', 'LB', 'IT', 'FITB', 'DXC', 'ROST', 'PPL', 'MAS', 'ALL', 'LEN', 'IVZ', 'ANSS', 'EXC', 'UNM', 'LH', 'LUK', 'GGP', 'HBI', 'WYN', 'MCO', 'IPG', 'MAC', 'GIS', 'EL', 'PCAR', 'UHS', 'VRSK', 'MKC', 'VAR', 'STT', 'TRV', 'AWK', 'CMA', 'DPS', 'LNC', 'XRX', 'RF', 'FLIR', 'VTR', 'CTAS', 'CHK', 'FTI', 'RJF', 'MSI', 'OMC', 'EXR', 'SCG', 'HRS', 'SNPS', 'VNO', 'D', 'WAT', 'DUK', 'DRI', 'JBHT', 'WM', 'LKQ', 'DISCK', 'AVB', 'LLL', 'SRCL', 'CCI', 'ROP', 'FBHS', 'IRM', 'BWA', 'HST', 'MAA', 'JEC', 'COO', 'AME', 'SJM', 'CA', 'FRT', 'FOX', 'A', 'RHI', 'MTB', 'PKG', 'DVA', 'EIX', 'ALLE', 'SIG', 'SRE', 'AEP', 'PKI', 'HSIC', 'WLTW', 'PGR', 'AON', 'FAST', 'RMD', 'COL', 'FTV', 'EQR', 'STI', 'NBL', 'RE', 'IFF', 'WU', 'AVY', 'FIS', 'BHF', 'TEL', 'DGX', 'WRK', 'PNR', 'HBAN', 'PEG', 'CNP', 'XYL', 'AEE', 'FLS', 'PFG', 'BEN', 'TSS', 'CSRA', 'DISCA', 'L', 'NI', 'ESS', 'ARE', 'EXPD', 'AES', 'DLPH', 'RSG', 'XEL', 'ZION', 'BLL', 'APH', 'BF.B', 'CMS', 'LEG', 'XRAY', 'SLG', 'HCP', 'PBCT', 'WEC', 'ES', 'CFG', 'PLD', 'AIV', 'LNT', 'AOS', 'KIM', 'CHD', 'DTE', 'MMC', 'NAVI', 'TMK', 'UDR', 'PNW', 'AJG', 'DRE', 'REG', 'NWSA', 'INFO', 'NWS', 'DGX', 'BK'), value='AMZN'), Output()), _dom_classes=('widget-interact',))

In [26]:
def graphPutsByTicker(tickerSymbol):
    opPriceDic = priceToArray(getPuts(getOptionPricesByTicker(tickerSymbol)))
    numOfDays = sparkSession.sql("SELECT NumberOfDays FROM optionData WHERE Ticker Like '" + tickerSymbol + "'").collect()
    numberOfDays = getNumberDaysArray(numOfDays)
    for key, values in opPriceDic.items():
        plt.plot(numberOfDays, values)
    plt.xlabel('Days to Expiration')
    plt.ylabel('Option Prices')
    plt.title('Put Options for '+ tickerSymbol)
    plt.show()

In [27]:
interact(graphPutsByTicker, tickerSymbol=tickerSymbols);

interactive(children=(Dropdown(description='tickerSymbol', options=('AMZN', 'GOOGL', 'ROK', 'GOOG', 'PCLN', 'MCHP', 'SPG', 'FDX', 'NTRS', 'AZO', 'NFLX', 'AAPL', 'CMG', 'CB', 'BA', 'AMAT', 'MU', 'VMC', 'NVDA', 'STX', 'CRM', 'WMT', 'CELG', 'AGN', 'GILD', 'BIIB', 'ADSK', 'CHTR', 'GPN', 'TGT', 'MSFT', 'MHK', 'NTAP', 'C', 'LOW', 'ABC', 'WDC', 'ISRG', 'DAL', 'JPM', 'LRCX', 'LMT', 'REGN', 'INTC', 'HCN', 'PYPL', 'BBY', 'FB', 'VLO', 'CAT', 'UAL', 'ULTA', 'GM', 'QCOM', 'AVGO', 'XOM', 'GS', 'ILMN', 'WYNN', 'MO', 'V', 'BLK', 'UNP', 'ORCL', 'ATVI', 'JNJ', 'SCHW', 'CMCSA', 'KSS', 'ABBV', 'RHT', 'SLB', 'ALGN', 'CVX', 'CLX', 'ADI', 'MGM', 'DE', 'COF', 'NKE', 'DLTR', 'PG', 'DG', 'NEM', 'AXP', 'CVS', 'DVN', 'WFC', 'CBOE', 'MCD', 'HD', 'EA', 'ESRX', 'TXN', 'ADBE', 'MNST', 'UPS', 'NOC', 'SHW', 'GE', 'URI', 'VZ', 'RCL', 'HCA', 'COST', 'CSCO', 'MMM', 'UNH', 'XLNX', 'DWDP', 'UTX', 'F', 'NSC', 'DIS', 'EXPE', 'TRIP', 'DISH', 'IBM', 'GPS', 'LUV', 'HAL', 'APC', 'CBS', 'AIG', 'BMY', 'AAL', 'NWL', 'PXD', 'CAH', 'APA', 'SWKS', 'MDLZ', 'KO', 'AMGN', 'HON', 'SYF', 'BAC', 'MOS', 'HAS', 'SBUX', 'ANTM', 'GWW', 'M', 'NUE', 'EFX', 'T', 'PEP', 'AKAM', 'PM', 'ANDV', 'EBAY', 'HUM', 'MAR', 'WBA', 'MRK', 'GRMN', 'KSU', 'LYB', 'TSCO', 'EOG', 'AMT', 'HES', 'ABT', 'AMD', 'TIF', 'CMI', 'INCY', 'KR', 'IP', 'MS', 'MDT', 'CME', 'OXY', 'PFE', 'MPC', 'KORS', 'BDX', 'PSX', 'FCX', 'ALXN', 'EQIX', 'CSX', 'GD', 'PCG', 'DFS', 'CCL', 'ETFC', 'COP', 'WHR', 'MET', 'DOV', 'CTL', 'ADM', 'HOG', 'GLW', 'EMR', 'ITW', 'NRG', 'DHI', 'MA', 'TROW', 'VFC', 'TWX', 'LLY', 'FFIV', 'CI', 'ETN', 'TAP', 'USB', 'KMI', 'RRC', 'HSY', 'CF', 'RL', 'MYL', 'VIAB', 'CTSH', 'YUM', 'AFL', 'SPGI', 'MCK', 'VRTX', 'FL', 'KMX', 'NOV', 'STZ', 'MON', 'TXT', 'TMO', 'CL', 'NFX', 'TSN', 'BK', 'ZTS', 'GPC', 'EW', 'CNC', 'HOLX', 'CXO', 'KMB', 'OKE', 'DHR', 'ADP', 'PPG', 'HP', 'SWK', 'ED', 'PSA', 'AAP', 'PH', 'BAX', 'XL', 'PDCO', 'HIG', 'MRO', 'BBT', 'ALB', 'FLR', 'CBG', 'GT', 'SYK', 'HRL', 'KLAC', 'PX', 'PRGO', 'PRU', 'WMB', 'MAT', 'PHM', 'ADS', 'HPQ', 'BHGE', 'ECL', 'AMG', 'BSX', 'PVH', 'IR', 'UAA', 'TPR', 'AMP', 'ACN', 'BRK.B', 'SNA', 'APD', 'QRVO', 'NCLH', 'SBAC', 'CTXS', 'EMN', 'ORLY', 'KEY', 'NDAQ', 'CAG', 'ARNC', 'AET', 'JNPR', 'RTN', 'SO', 'ICE', 'CERN', 'HPE', 'EQT', 'PAYX', 'FISV', 'VRSN', 'DLR', 'JCI', 'XEC', 'SYMC', 'COTY', 'K', 'COG', 'CPB', 'MTD', 'INTU', 'AYI', 'CDNS', 'AIZ', 'MLM', 'KHC', 'EVHC', 'TDG', 'NLSN', 'WY', 'UA', 'ZBH', 'NEE', 'ETR', 'CINF', 'FOXA', 'ALK', 'SEE', 'SNI', 'BXP', 'TJX', 'PWR', 'PNC', 'IDXX', 'HLT', 'JWN', 'FMC', 'O', 'CHRW', 'SYY', 'HRB', 'FE', 'LB', 'IT', 'FITB', 'DXC', 'ROST', 'PPL', 'MAS', 'ALL', 'LEN', 'IVZ', 'ANSS', 'EXC', 'UNM', 'LH', 'LUK', 'GGP', 'HBI', 'WYN', 'MCO', 'IPG', 'MAC', 'GIS', 'EL', 'PCAR', 'UHS', 'VRSK', 'MKC', 'VAR', 'STT', 'TRV', 'AWK', 'CMA', 'DPS', 'LNC', 'XRX', 'RF', 'FLIR', 'VTR', 'CTAS', 'CHK', 'FTI', 'RJF', 'MSI', 'OMC', 'EXR', 'SCG', 'HRS', 'SNPS', 'VNO', 'D', 'WAT', 'DUK', 'DRI', 'JBHT', 'WM', 'LKQ', 'DISCK', 'AVB', 'LLL', 'SRCL', 'CCI', 'ROP', 'FBHS', 'IRM', 'BWA', 'HST', 'MAA', 'JEC', 'COO', 'AME', 'SJM', 'CA', 'FRT', 'FOX', 'A', 'RHI', 'MTB', 'PKG', 'DVA', 'EIX', 'ALLE', 'SIG', 'SRE', 'AEP', 'PKI', 'HSIC', 'WLTW', 'PGR', 'AON', 'FAST', 'RMD', 'COL', 'FTV', 'EQR', 'STI', 'NBL', 'RE', 'IFF', 'WU', 'AVY', 'FIS', 'BHF', 'TEL', 'DGX', 'WRK', 'PNR', 'HBAN', 'PEG', 'CNP', 'XYL', 'AEE', 'FLS', 'PFG', 'BEN', 'TSS', 'CSRA', 'DISCA', 'L', 'NI', 'ESS', 'ARE', 'EXPD', 'AES', 'DLPH', 'RSG', 'XEL', 'ZION', 'BLL', 'APH', 'BF.B', 'CMS', 'LEG', 'XRAY', 'SLG', 'HCP', 'PBCT', 'WEC', 'ES', 'CFG', 'PLD', 'AIV', 'LNT', 'AOS', 'KIM', 'CHD', 'DTE', 'MMC', 'NAVI', 'TMK', 'UDR', 'PNW', 'AJG', 'DRE', 'REG', 'NWSA', 'INFO', 'NWS', 'DGX', 'BK'), value='AMZN'), Output()), _dom_classes=('widget-interact',))

In [83]:
# def getPricesByStrike2(optionPrices, strike): # Think i can remove
#     allPrices = []
#     for i in optionPrices:
#         if i!= None and i['StrikePrice'] == strike:
#             allPrices.append(i['Put'])
#     for i in allPrices:
#         if i is None:
#             allPrices.remove(i)
#     for i in allPrices: # for some reason have to run twice as it skips last None in above 
#         if i is None:
#             allPrices.remove(i)
#     return allPrices


In [84]:
def priceToDicForStrikeGraph(opPrices):
    prices = {}
    count = 1
    for i in opPrices:
        priceArr = []
        for x in i:
            if x is not None:
                priceArr.append(x)
        prices['opt'+str(count)] = (priceArr.copy())
        count = count + 1
    return prices

In [138]:
def graphPutsByTickerWithStrike(strikePrice, tickerSymbol):
    prices = getOptionPricesByTicker(tickerSymbol)
    putPrices = getPuts2(prices)
    rates = getRiskRateForGraph(tickerSymbol)
    pp = getPricesByStrike( putPrices,strikePrice)
    opPriceDic = priceToDicForStrikeGraph(pp)
    numOfDays = sparkSession.sql("SELECT NumberOfDays FROM optionData WHERE Ticker Like '" +
                                 tickerSymbol + "'").collect()
    numberOfDays = getNumberDaysArray(numOfDays)
    count = 0 # to get correct rates for graph pointer texts
    for key, values in opPriceDic.items():
        textXPoint = numberOfDays[len(numberOfDays)-1] + 0.4 # cant do this in .annotate(xytext) 
        plt.plot(numberOfDays, values)
        plt.annotate('Risk rate: '+str(rates[count]), xy=(numberOfDays[len(numberOfDays)-1], values[len(values)-1]), 
             xytext=(textXPoint, values[len(values)-1]),
        arrowprops=dict(arrowstyle='wedge'), bbox=dict(boxstyle="round", fc="w"),
        )
        count = count +1
    plt.xlabel('Days to Expiration')
    plt.ylabel('Option Prices')
    plt.title('Put Options for '+ tickerSymbol + ' with Strike Price €' + str(strikePrice))
    plt.show()

In [139]:
def passTickerforPutStrike(tickerSymbol):
    strikePrices = sparkSession.sql("SELECT Prices['StrikePrice'] FROM optionData WHERE Ticker Like '"+ 
                                    tickerSymbol+"'").collect() # all use the same rates
    strikePrices = getStrikePriceUnique(strikePrices)
    interact(graphPutsByTickerWithStrike,strikePrice = strikePrices, tickerSymbol = fixed(tickerSymbol)); # Fixed removes extra interaction box

In [140]:

interact(passTickerforPutStrike,tickerSymbol=tickerSymbols);

interactive(children=(Dropdown(description='tickerSymbol', options=('AMZN', 'GOOGL', 'ROK', 'GOOG', 'PCLN', 'MCHP', 'SPG', 'FDX', 'NTRS', 'AZO', 'NFLX', 'AAPL', 'CMG', 'CB', 'BA', 'AMAT', 'MU', 'VMC', 'NVDA', 'STX', 'CRM', 'WMT', 'CELG', 'AGN', 'GILD', 'BIIB', 'ADSK', 'CHTR', 'GPN', 'TGT', 'MSFT', 'MHK', 'NTAP', 'C', 'LOW', 'ABC', 'WDC', 'ISRG', 'DAL', 'JPM', 'LRCX', 'LMT', 'REGN', 'INTC', 'HCN', 'PYPL', 'BBY', 'FB', 'VLO', 'CAT', 'UAL', 'ULTA', 'GM', 'QCOM', 'AVGO', 'XOM', 'GS', 'ILMN', 'WYNN', 'MO', 'V', 'BLK', 'UNP', 'ORCL', 'ATVI', 'JNJ', 'SCHW', 'CMCSA', 'KSS', 'ABBV', 'RHT', 'SLB', 'ALGN', 'CVX', 'CLX', 'ADI', 'MGM', 'DE', 'COF', 'NKE', 'DLTR', 'PG', 'DG', 'NEM', 'AXP', 'CVS', 'DVN', 'WFC', 'CBOE', 'MCD', 'HD', 'EA', 'ESRX', 'TXN', 'ADBE', 'MNST', 'UPS', 'NOC', 'SHW', 'GE', 'URI', 'VZ', 'RCL', 'HCA', 'COST', 'CSCO', 'MMM', 'UNH', 'XLNX', 'DWDP', 'UTX', 'F', 'NSC', 'DIS', 'EXPE', 'TRIP', 'DISH', 'IBM', 'GPS', 'LUV', 'HAL', 'APC', 'CBS', 'AIG', 'BMY', 'AAL', 'NWL', 'PXD', 'CAH', 'APA', 'SWKS', 'MDLZ', 'KO', 'AMGN', 'HON', 'SYF', 'BAC', 'MOS', 'HAS', 'SBUX', 'ANTM', 'GWW', 'M', 'NUE', 'EFX', 'T', 'PEP', 'AKAM', 'PM', 'ANDV', 'EBAY', 'HUM', 'MAR', 'WBA', 'MRK', 'GRMN', 'KSU', 'LYB', 'TSCO', 'EOG', 'AMT', 'HES', 'ABT', 'AMD', 'TIF', 'CMI', 'INCY', 'KR', 'IP', 'MS', 'MDT', 'CME', 'OXY', 'PFE', 'MPC', 'KORS', 'BDX', 'PSX', 'FCX', 'ALXN', 'EQIX', 'CSX', 'GD', 'PCG', 'DFS', 'CCL', 'ETFC', 'COP', 'WHR', 'MET', 'DOV', 'CTL', 'ADM', 'HOG', 'GLW', 'EMR', 'ITW', 'NRG', 'DHI', 'MA', 'TROW', 'VFC', 'TWX', 'LLY', 'FFIV', 'CI', 'ETN', 'TAP', 'USB', 'KMI', 'RRC', 'HSY', 'CF', 'RL', 'MYL', 'VIAB', 'CTSH', 'YUM', 'AFL', 'SPGI', 'MCK', 'VRTX', 'FL', 'KMX', 'NOV', 'STZ', 'MON', 'TXT', 'TMO', 'CL', 'NFX', 'TSN', 'BK', 'ZTS', 'GPC', 'EW', 'CNC', 'HOLX', 'CXO', 'KMB', 'OKE', 'DHR', 'ADP', 'PPG', 'HP', 'SWK', 'ED', 'PSA', 'AAP', 'PH', 'BAX', 'XL', 'PDCO', 'HIG', 'MRO', 'BBT', 'ALB', 'FLR', 'CBG', 'GT', 'SYK', 'HRL', 'KLAC', 'PX', 'PRGO', 'PRU', 'WMB', 'MAT', 'PHM', 'ADS', 'HPQ', 'BHGE', 'ECL', 'AMG', 'BSX', 'PVH', 'IR', 'UAA', 'TPR', 'AMP', 'ACN', 'BRK.B', 'SNA', 'APD', 'QRVO', 'NCLH', 'SBAC', 'CTXS', 'EMN', 'ORLY', 'KEY', 'NDAQ', 'CAG', 'ARNC', 'AET', 'JNPR', 'RTN', 'SO', 'ICE', 'CERN', 'HPE', 'EQT', 'PAYX', 'FISV', 'VRSN', 'DLR', 'JCI', 'XEC', 'SYMC', 'COTY', 'K', 'COG', 'CPB', 'MTD', 'INTU', 'AYI', 'CDNS', 'AIZ', 'MLM', 'KHC', 'EVHC', 'TDG', 'NLSN', 'WY', 'UA', 'ZBH', 'NEE', 'ETR', 'CINF', 'FOXA', 'ALK', 'SEE', 'SNI', 'BXP', 'TJX', 'PWR', 'PNC', 'IDXX', 'HLT', 'JWN', 'FMC', 'O', 'CHRW', 'SYY', 'HRB', 'FE', 'LB', 'IT', 'FITB', 'DXC', 'ROST', 'PPL', 'MAS', 'ALL', 'LEN', 'IVZ', 'ANSS', 'EXC', 'UNM', 'LH', 'LUK', 'GGP', 'HBI', 'WYN', 'MCO', 'IPG', 'MAC', 'GIS', 'EL', 'PCAR', 'UHS', 'VRSK', 'MKC', 'VAR', 'STT', 'TRV', 'AWK', 'CMA', 'DPS', 'LNC', 'XRX', 'RF', 'FLIR', 'VTR', 'CTAS', 'CHK', 'FTI', 'RJF', 'MSI', 'OMC', 'EXR', 'SCG', 'HRS', 'SNPS', 'VNO', 'D', 'WAT', 'DUK', 'DRI', 'JBHT', 'WM', 'LKQ', 'DISCK', 'AVB', 'LLL', 'SRCL', 'CCI', 'ROP', 'FBHS', 'IRM', 'BWA', 'HST', 'MAA', 'JEC', 'COO', 'AME', 'SJM', 'CA', 'FRT', 'FOX', 'A', 'RHI', 'MTB', 'PKG', 'DVA', 'EIX', 'ALLE', 'SIG', 'SRE', 'AEP', 'PKI', 'HSIC', 'WLTW', 'PGR', 'AON', 'FAST', 'RMD', 'COL', 'FTV', 'EQR', 'STI', 'NBL', 'RE', 'IFF', 'WU', 'AVY', 'FIS', 'BHF', 'TEL', 'DGX', 'WRK', 'PNR', 'HBAN', 'PEG', 'CNP', 'XYL', 'AEE', 'FLS', 'PFG', 'BEN', 'TSS', 'CSRA', 'DISCA', 'L', 'NI', 'ESS', 'ARE', 'EXPD', 'AES', 'DLPH', 'RSG', 'XEL', 'ZION', 'BLL', 'APH', 'BF.B', 'CMS', 'LEG', 'XRAY', 'SLG', 'HCP', 'PBCT', 'WEC', 'ES', 'CFG', 'PLD', 'AIV', 'LNT', 'AOS', 'KIM', 'CHD', 'DTE', 'MMC', 'NAVI', 'TMK', 'UDR', 'PNW', 'AJG', 'DRE', 'REG', 'NWSA', 'INFO', 'NWS', 'DGX', 'BK'), value='AMZN'), Output()), _dom_classes=('widget-interact',))

In [225]:
def getPricesByStrikeCall(optionPrices, strike):
    allPrices = []
    for i in optionPrices:
        if i!= None and i['StrikePrice'] == strike:
            allPrices.append(i['Call'])
    for i in allPrices:
        if i is None:
            allPrices.remove(i)
    for i in allPrices: # for some reason have to run twice as it skips last None in above 
        if i is None:
            allPrices.remove(i)
    return allPrices


def graphCallsByTickerWithStrike(strikePrice, tickerSymbol):
    prices = getOptionPricesByTicker(tickerSymbol)
    callPrices = getCalls(prices)
    rates = getRiskRateForGraph(tickerSymbol)
    pp = getPricesByStrikeCall( callPrices,strikePrice)
    opPriceDic = priceToArray2(pp)
    numOfDays = sparkSession.sql("SELECT NumberOfDays FROM optionData WHERE Ticker Like '" + tickerSymbol + "'").collect()
    numberOfDays = getNumberDaysArray(numOfDays)
    count = 0 # to get correct rates for graph pointer texts
    for key, values in opPriceDic.items():
        textXPoint = numberOfDays[len(numberOfDays)-1] + 1  # cant do this in .annotate(xytext) 
        plt.plot(numberOfDays, values)
        plt.annotate('Risk rate: '+str(rates[count]), xy=(numberOfDays[len(numberOfDays)-1], values[len(values)-1]), 
                     xytext=(textXPoint, values[len(values)-1]),
            arrowprops=dict(arrowstyle='wedge'), bbox=dict(boxstyle="round", fc="w"),
            )
        count = count +1
    plt.xlabel('Days to Expiration')
    plt.ylabel('Option Prices')
    plt.title('Call Options for '+ tickerSymbol + ' with Strike Price €' + str(strikePrice))
    plt.ticklabel_format(useOffset=False)
    plt.show()
    
def passTickerforCallStrike(tickerSymbol):
    strikePrices = sparkSession.sql("SELECT Prices['StrikePrice'] FROM optionData WHERE Ticker Like '"+ tickerSymbol+"'").collect() # all use the same rates
    strikePrices = getStrikePriceUnique(strikePrices)
    interact(graphCallsByTickerWithStrike, strikePrice = strikePrices, tickerSymbol = fixed(tickerSymbol)); # Fixed removes extra interaction box);

In [226]:
interact(passTickerforCallStrike, tickerSymbol=tickerSymbols);

interactive(children=(Dropdown(description='tickerSymbol', options=('AMZN', 'GOOGL', 'ROK', 'GOOG', 'PCLN', 'MCHP', 'SPG', 'FDX', 'NTRS', 'AZO', 'NFLX', 'AAPL', 'CMG', 'CB', 'BA', 'AMAT', 'MU', 'VMC', 'NVDA', 'STX', 'CRM', 'WMT', 'CELG', 'AGN', 'GILD', 'BIIB', 'ADSK', 'CHTR', 'GPN', 'TGT', 'MSFT', 'MHK', 'NTAP', 'C', 'LOW', 'ABC', 'WDC', 'ISRG', 'DAL', 'JPM', 'LRCX', 'LMT', 'REGN', 'INTC', 'HCN', 'PYPL', 'BBY', 'FB', 'VLO', 'CAT', 'UAL', 'ULTA', 'GM', 'QCOM', 'AVGO', 'XOM', 'GS', 'ILMN', 'WYNN', 'MO', 'V', 'BLK', 'UNP', 'ORCL', 'ATVI', 'JNJ', 'SCHW', 'CMCSA', 'KSS', 'ABBV', 'RHT', 'SLB', 'ALGN', 'CVX', 'CLX', 'ADI', 'MGM', 'DE', 'COF', 'NKE', 'DLTR', 'PG', 'DG', 'NEM', 'AXP', 'CVS', 'DVN', 'WFC', 'CBOE', 'MCD', 'HD', 'EA', 'ESRX', 'TXN', 'ADBE', 'MNST', 'UPS', 'NOC', 'SHW', 'GE', 'URI', 'VZ', 'RCL', 'HCA', 'COST', 'CSCO', 'MMM', 'UNH', 'XLNX', 'DWDP', 'UTX', 'F', 'NSC', 'DIS', 'EXPE', 'TRIP', 'DISH', 'IBM', 'GPS', 'LUV', 'HAL', 'APC', 'CBS', 'AIG', 'BMY', 'AAL', 'NWL', 'PXD', 'CAH', 'APA', 'SWKS', 'MDLZ', 'KO', 'AMGN', 'HON', 'SYF', 'BAC', 'MOS', 'HAS', 'SBUX', 'ANTM', 'GWW', 'M', 'NUE', 'EFX', 'T', 'PEP', 'AKAM', 'PM', 'ANDV', 'EBAY', 'HUM', 'MAR', 'WBA', 'MRK', 'GRMN', 'KSU', 'LYB', 'TSCO', 'EOG', 'AMT', 'HES', 'ABT', 'AMD', 'TIF', 'CMI', 'INCY', 'KR', 'IP', 'MS', 'MDT', 'CME', 'OXY', 'PFE', 'MPC', 'KORS', 'BDX', 'PSX', 'FCX', 'ALXN', 'EQIX', 'CSX', 'GD', 'PCG', 'DFS', 'CCL', 'ETFC', 'COP', 'WHR', 'MET', 'DOV', 'CTL', 'ADM', 'HOG', 'GLW', 'EMR', 'ITW', 'NRG', 'DHI', 'MA', 'TROW', 'VFC', 'TWX', 'LLY', 'FFIV', 'CI', 'ETN', 'TAP', 'USB', 'KMI', 'RRC', 'HSY', 'CF', 'RL', 'MYL', 'VIAB', 'CTSH', 'YUM', 'AFL', 'SPGI', 'MCK', 'VRTX', 'FL', 'KMX', 'NOV', 'STZ', 'MON', 'TXT', 'TMO', 'CL', 'NFX', 'TSN', 'BK', 'ZTS', 'GPC', 'EW', 'CNC', 'HOLX', 'CXO', 'KMB', 'OKE', 'DHR', 'ADP', 'PPG', 'HP', 'SWK', 'ED', 'PSA', 'AAP', 'PH', 'BAX', 'XL', 'PDCO', 'HIG', 'MRO', 'BBT', 'ALB', 'FLR', 'CBG', 'GT', 'SYK', 'HRL', 'KLAC', 'PX', 'PRGO', 'PRU', 'WMB', 'MAT', 'PHM', 'ADS', 'HPQ', 'BHGE', 'ECL', 'AMG', 'BSX', 'PVH', 'IR', 'UAA', 'TPR', 'AMP', 'ACN', 'BRK.B', 'SNA', 'APD', 'QRVO', 'NCLH', 'SBAC', 'CTXS', 'EMN', 'ORLY', 'KEY', 'NDAQ', 'CAG', 'ARNC', 'AET', 'JNPR', 'RTN', 'SO', 'ICE', 'CERN', 'HPE', 'EQT', 'PAYX', 'FISV', 'VRSN', 'DLR', 'JCI', 'XEC', 'SYMC', 'COTY', 'K', 'COG', 'CPB', 'MTD', 'INTU', 'AYI', 'CDNS', 'AIZ', 'MLM', 'KHC', 'EVHC', 'TDG', 'NLSN', 'WY', 'UA', 'ZBH', 'NEE', 'ETR', 'CINF', 'FOXA', 'ALK', 'SEE', 'SNI', 'BXP', 'TJX', 'PWR', 'PNC', 'IDXX', 'HLT', 'JWN', 'FMC', 'O', 'CHRW', 'SYY', 'HRB', 'FE', 'LB', 'IT', 'FITB', 'DXC', 'ROST', 'PPL', 'MAS', 'ALL', 'LEN', 'IVZ', 'ANSS', 'EXC', 'UNM', 'LH', 'LUK', 'GGP', 'HBI', 'WYN', 'MCO', 'IPG', 'MAC', 'GIS', 'EL', 'PCAR', 'UHS', 'VRSK', 'MKC', 'VAR', 'STT', 'TRV', 'AWK', 'CMA', 'DPS', 'LNC', 'XRX', 'RF', 'FLIR', 'VTR', 'CTAS', 'CHK', 'FTI', 'RJF', 'MSI', 'OMC', 'EXR', 'SCG', 'HRS', 'SNPS', 'VNO', 'D', 'WAT', 'DUK', 'DRI', 'JBHT', 'WM', 'LKQ', 'DISCK', 'AVB', 'LLL', 'SRCL', 'CCI', 'ROP', 'FBHS', 'IRM', 'BWA', 'HST', 'MAA', 'JEC', 'COO', 'AME', 'SJM', 'CA', 'FRT', 'FOX', 'A', 'RHI', 'MTB', 'PKG', 'DVA', 'EIX', 'ALLE', 'SIG', 'SRE', 'AEP', 'PKI', 'HSIC', 'WLTW', 'PGR', 'AON', 'FAST', 'RMD', 'COL', 'FTV', 'EQR', 'STI', 'NBL', 'RE', 'IFF', 'WU', 'AVY', 'FIS', 'BHF', 'TEL', 'DGX', 'WRK', 'PNR', 'HBAN', 'PEG', 'CNP', 'XYL', 'AEE', 'FLS', 'PFG', 'BEN', 'TSS', 'CSRA', 'DISCA', 'L', 'NI', 'ESS', 'ARE', 'EXPD', 'AES', 'DLPH', 'RSG', 'XEL', 'ZION', 'BLL', 'APH', 'BF.B', 'CMS', 'LEG', 'XRAY', 'SLG', 'HCP', 'PBCT', 'WEC', 'ES', 'CFG', 'PLD', 'AIV', 'LNT', 'AOS', 'KIM', 'CHD', 'DTE', 'MMC', 'NAVI', 'TMK', 'UDR', 'PNW', 'AJG', 'DRE', 'REG', 'NWSA', 'INFO', 'NWS', 'DGX', 'BK'), value='AMZN'), Output()), _dom_classes=('widget-interact',))

In [223]:
def getCalls2(optionPrices):
    callPrices = []
    for i in optionPrices:
        if i['Call'] != None:
            callPrices.append(i[0])
    return callPrices

In [207]:
def getPricesByRiskFreeRate(optionPrices, RiskFreeRate):
    allPrices = []
    for i in optionPrices[0][0]:
        if i['RiskFreeRate'] == RiskFreeRate:
            allPrices.append(i)
    return allPrices

In [224]:
def graphCallByTickerWithRate(tickerSymbol, rate):
    prices = getOptionPricesByTicker(tickerSymbol)
    ratePrices = getPricesByRiskFreeRate(prices,rate)
    prices = getCalls2(ratePrices)
    opPriceDic = priceToArray2(prices)
    numOfDays = sparkSession.sql("SELECT NumberOfDays FROM optionData WHERE Ticker Like '" + tickerSymbol + "'").collect()
    numberOfDays = getNumberDaysArray(numOfDays)
    for key, values in opPriceDic.items(): 
        plt.plot(numberOfDays, values)
    plt.xlabel('Days to Expiration')
    plt.ylabel('Option Prices')
    plt.title('Call Options for '+ tickerSymbol + ' with risk free rate ' + str(rate))
    plt.ticklabel_format(style='plain')
    plt.show()
    
    
# numOfDays = sparkSession.sql("SELECT NumberOfDays FROM optionData WHERE Ticker Like '" + tickerSymbol + "'").collect()
# numberOfDays = getNumberDaysArray(numOfDays)
# plt.plot(numberOfDays, opPriceDic)
# plt.xlabel('Days to Expiration')
# plt.ylabel('Option Prices')
# plt.title('Call Options for '+ tickerSymbol + ' with risk free rate ' + str(rate))
# plt.show()

In [220]:
def passRateCall(Ticker):
    riskFreeRates = getRiskRateForGraph(Ticker)
    interact(graphCallByTickerWithRate,  tickerSymbol=fixed(Ticker), rate = riskFreeRates);

In [221]:
interact(passRateCall, Ticker=tickerSymbols);

interactive(children=(Dropdown(description='Ticker', options=('AMZN', 'GOOGL', 'ROK', 'GOOG', 'PCLN', 'MCHP', 'SPG', 'FDX', 'NTRS', 'AZO', 'NFLX', 'AAPL', 'CMG', 'CB', 'BA', 'AMAT', 'MU', 'VMC', 'NVDA', 'STX', 'CRM', 'WMT', 'CELG', 'AGN', 'GILD', 'BIIB', 'ADSK', 'CHTR', 'GPN', 'TGT', 'MSFT', 'MHK', 'NTAP', 'C', 'LOW', 'ABC', 'WDC', 'ISRG', 'DAL', 'JPM', 'LRCX', 'LMT', 'REGN', 'INTC', 'HCN', 'PYPL', 'BBY', 'FB', 'VLO', 'CAT', 'UAL', 'ULTA', 'GM', 'QCOM', 'AVGO', 'XOM', 'GS', 'ILMN', 'WYNN', 'MO', 'V', 'BLK', 'UNP', 'ORCL', 'ATVI', 'JNJ', 'SCHW', 'CMCSA', 'KSS', 'ABBV', 'RHT', 'SLB', 'ALGN', 'CVX', 'CLX', 'ADI', 'MGM', 'DE', 'COF', 'NKE', 'DLTR', 'PG', 'DG', 'NEM', 'AXP', 'CVS', 'DVN', 'WFC', 'CBOE', 'MCD', 'HD', 'EA', 'ESRX', 'TXN', 'ADBE', 'MNST', 'UPS', 'NOC', 'SHW', 'GE', 'URI', 'VZ', 'RCL', 'HCA', 'COST', 'CSCO', 'MMM', 'UNH', 'XLNX', 'DWDP', 'UTX', 'F', 'NSC', 'DIS', 'EXPE', 'TRIP', 'DISH', 'IBM', 'GPS', 'LUV', 'HAL', 'APC', 'CBS', 'AIG', 'BMY', 'AAL', 'NWL', 'PXD', 'CAH', 'APA', 'SWKS', 'MDLZ', 'KO', 'AMGN', 'HON', 'SYF', 'BAC', 'MOS', 'HAS', 'SBUX', 'ANTM', 'GWW', 'M', 'NUE', 'EFX', 'T', 'PEP', 'AKAM', 'PM', 'ANDV', 'EBAY', 'HUM', 'MAR', 'WBA', 'MRK', 'GRMN', 'KSU', 'LYB', 'TSCO', 'EOG', 'AMT', 'HES', 'ABT', 'AMD', 'TIF', 'CMI', 'INCY', 'KR', 'IP', 'MS', 'MDT', 'CME', 'OXY', 'PFE', 'MPC', 'KORS', 'BDX', 'PSX', 'FCX', 'ALXN', 'EQIX', 'CSX', 'GD', 'PCG', 'DFS', 'CCL', 'ETFC', 'COP', 'WHR', 'MET', 'DOV', 'CTL', 'ADM', 'HOG', 'GLW', 'EMR', 'ITW', 'NRG', 'DHI', 'MA', 'TROW', 'VFC', 'TWX', 'LLY', 'FFIV', 'CI', 'ETN', 'TAP', 'USB', 'KMI', 'RRC', 'HSY', 'CF', 'RL', 'MYL', 'VIAB', 'CTSH', 'YUM', 'AFL', 'SPGI', 'MCK', 'VRTX', 'FL', 'KMX', 'NOV', 'STZ', 'MON', 'TXT', 'TMO', 'CL', 'NFX', 'TSN', 'BK', 'ZTS', 'GPC', 'EW', 'CNC', 'HOLX', 'CXO', 'KMB', 'OKE', 'DHR', 'ADP', 'PPG', 'HP', 'SWK', 'ED', 'PSA', 'AAP', 'PH', 'BAX', 'XL', 'PDCO', 'HIG', 'MRO', 'BBT', 'ALB', 'FLR', 'CBG', 'GT', 'SYK', 'HRL', 'KLAC', 'PX', 'PRGO', 'PRU', 'WMB', 'MAT', 'PHM', 'ADS', 'HPQ', 'BHGE', 'ECL', 'AMG', 'BSX', 'PVH', 'IR', 'UAA', 'TPR', 'AMP', 'ACN', 'BRK.B', 'SNA', 'APD', 'QRVO', 'NCLH', 'SBAC', 'CTXS', 'EMN', 'ORLY', 'KEY', 'NDAQ', 'CAG', 'ARNC', 'AET', 'JNPR', 'RTN', 'SO', 'ICE', 'CERN', 'HPE', 'EQT', 'PAYX', 'FISV', 'VRSN', 'DLR', 'JCI', 'XEC', 'SYMC', 'COTY', 'K', 'COG', 'CPB', 'MTD', 'INTU', 'AYI', 'CDNS', 'AIZ', 'MLM', 'KHC', 'EVHC', 'TDG', 'NLSN', 'WY', 'UA', 'ZBH', 'NEE', 'ETR', 'CINF', 'FOXA', 'ALK', 'SEE', 'SNI', 'BXP', 'TJX', 'PWR', 'PNC', 'IDXX', 'HLT', 'JWN', 'FMC', 'O', 'CHRW', 'SYY', 'HRB', 'FE', 'LB', 'IT', 'FITB', 'DXC', 'ROST', 'PPL', 'MAS', 'ALL', 'LEN', 'IVZ', 'ANSS', 'EXC', 'UNM', 'LH', 'LUK', 'GGP', 'HBI', 'WYN', 'MCO', 'IPG', 'MAC', 'GIS', 'EL', 'PCAR', 'UHS', 'VRSK', 'MKC', 'VAR', 'STT', 'TRV', 'AWK', 'CMA', 'DPS', 'LNC', 'XRX', 'RF', 'FLIR', 'VTR', 'CTAS', 'CHK', 'FTI', 'RJF', 'MSI', 'OMC', 'EXR', 'SCG', 'HRS', 'SNPS', 'VNO', 'D', 'WAT', 'DUK', 'DRI', 'JBHT', 'WM', 'LKQ', 'DISCK', 'AVB', 'LLL', 'SRCL', 'CCI', 'ROP', 'FBHS', 'IRM', 'BWA', 'HST', 'MAA', 'JEC', 'COO', 'AME', 'SJM', 'CA', 'FRT', 'FOX', 'A', 'RHI', 'MTB', 'PKG', 'DVA', 'EIX', 'ALLE', 'SIG', 'SRE', 'AEP', 'PKI', 'HSIC', 'WLTW', 'PGR', 'AON', 'FAST', 'RMD', 'COL', 'FTV', 'EQR', 'STI', 'NBL', 'RE', 'IFF', 'WU', 'AVY', 'FIS', 'BHF', 'TEL', 'DGX', 'WRK', 'PNR', 'HBAN', 'PEG', 'CNP', 'XYL', 'AEE', 'FLS', 'PFG', 'BEN', 'TSS', 'CSRA', 'DISCA', 'L', 'NI', 'ESS', 'ARE', 'EXPD', 'AES', 'DLPH', 'RSG', 'XEL', 'ZION', 'BLL', 'APH', 'BF.B', 'CMS', 'LEG', 'XRAY', 'SLG', 'HCP', 'PBCT', 'WEC', 'ES', 'CFG', 'PLD', 'AIV', 'LNT', 'AOS', 'KIM', 'CHD', 'DTE', 'MMC', 'NAVI', 'TMK', 'UDR', 'PNW', 'AJG', 'DRE', 'REG', 'NWSA', 'INFO', 'NWS', 'DGX', 'BK'), value='AMZN'), Output()), _dom_classes=('widget-interact',))

In [ ]:
# def graph(ticker):    
#     # Code for put by ticker
#     data = []
#     opPriceDic = priceToArray(getPuts(getOptionPricesByTicker(ticker)))
#     for key, values in opPriceDic.items():
#         trace = go.Scatter(
#             x = numberOfDays,
#             y = values,
#             name = 'option',
#             line = dict(
#                 width = 4)
#         )
#         data.append(trace)
#     # Edit the layout
#     layout = dict(title = 'Call prices for ' ,
#                   xaxis = dict(title = 'Days to Expiration'),
#                   yaxis = dict(title = 'Price '),
#                   )
#     fig = dict(data=data, layout=layout)
#     py.iplot(fig, filename='calls For ')

In [ ]:
# Code for calls by ticker
# data = []
# opPriceDic = priceToArray(getCalls(getOptionPricesByTicker('ABBV')))
# for key, values in opPriceDic.items():
#     trace = go.Scatter(
#         x = numberOfDays,
#         y = values,
#         name = 'option',
#         line = dict(
#             width = 4)
#     )
#     data.append(trace)
# # Edit the layout
# layout = dict(title = 'Call prices for ' ,
#               xaxis = dict(title = 'Days to Expiration'),
#               yaxis = dict(title = 'Price '),
#               )
# fig = dict(data=data, layout=layout)
# py.iplot(fig, filename='calls For ')

In [ ]:

# p.tools.set_credentials_file(username='lalor188', api_key='MQxJd3Rs9X5aKOZiDEgt') # Plotly # Not gonna be using to remove
# optionTicker = widgets.Dropdown( # not gonna work remove
# #     options=tickerSymbols,
# #     value=tickerSymbols[0],
# #     description='Option Ticker symbol:',
# # )
# # g = GraphWidget('https://plot.ly/~kevintest/1192/')

        
# # # def response2(change): # not gonna work remove
# # #     # print "using date"
# # #     if optionTicker.value:
# # #         filter_list = optionTicker.value
# # #         data = []
# # #         yData = []
# # #         opPriceDic = priceToArray(getPuts(getOptionPricesByTicker(filter_list)))
# # #         x = numberOfDays
# # #         for key, values in opPriceDic.items():
# # #             yData.append(values.copy())
# # #         g.restyle({'x': [numberOfDays],'y': [yData],  'type': 'line' })
# # #         g.relayout({'barmode': 'overlay', 'xaxis': {'title': 'Days to expiration'},
# # #                 'yaxis': {'title': 'Price ($)'}})
    
# # # optionTicker.observe(response2, names="value") # not gonna work remove all
# # # # container2 = widgets.HBox([optionTicker]) 
# # # # display(container2)
# # # # display(g)

In [105]:
# demo purposes To remove
# import plotly.plotly as py 
# import plotly.graph_objs as go 



# data = []
# opPriceDic = priceToArray(getPuts(getOptionPricesByTicker('ZTS')))
# for key, values in opPriceDic.items():
#     trace = go.Scatter(
#         x = numberOfDays,
#         y = values,
#         name = 'option',
#         line = dict(
#             width = 4)
#     )
#     data.append(trace)

# high_annotations=[dict(x='2016-03-01',
#                        y=5,
#                        xref='x', yref='y',
#                        text='High Average:<br>'+str(6),
#                        ax=0, ay=-40),
#                   dict(x=5,
#                        y=5,
#                        xref='x', yref='y',
#                        text='High Max:<br>'+str(6),
#                        ax=0, ay=-40)]
# low_annotations=[dict(x='2015-05-01',
#                       y=1,
#                       xref='x', yref='y',
#                       text='Low Average:<br>'+str(1),
#                       ax=0, ay=40),
#                  dict(x=1,
#                       y=1,
#                       xref='x', yref='y',
#                       text='Low Min:<br>'+str(1),
#                       ax=0, ay=40)]

# updatemenus = list([
#     dict(active=-1,
#          buttons=list([   
#             dict(label = 'MMM',
#                  method = 'update',
#                  args = [{'visible': [True, True, False, False]},
#                          {'title': 'Yahoo High',
#                           'annotations': high_annotations}]),
#             dict(label = 'Low',
#                  method = 'update',
#                  args = [{'visible': [False, False, True, True]},
#                          {'title': 'Yahoo Low',
#                           'annotations': low_annotations}]),
#             dict(label = 'Both',
#                  method = 'update',
#                  args = [{'visible': [True, True, True, True]},
#                          {'title': 'Yahoo',
#                           'annotations': high_annotations+low_annotations}]),
#             dict(label = 'Reset',
#                  method = 'update',
#                  args = [{'visible': [True, False, True, False]},
#                          {'title': 'Yahoo',
#                           'annotations': []}])
#         ]),
#     )
# ])

# layout = dict(title='Yahoo', showlegend=False,
#               updatemenus=updatemenus)

# fig = dict(data=data, layout=layout)
# py.iplot(fig, filename='update_dropdown')